In [ ]:
"""
    UAVSAR polarimetric InSAR data processing: UAVSAR data read, multi-looking, InSAR phase/coherence
    Based on Simon Zwieback's IO.py file
"""

In [2]:
import os
import numpy as np
from collections import namedtuple
from scipy import ndimage
from scipy.special import comb

In [3]:
def UAVSARReadQuadPolSLC(fnann, polarizations=['HH', 'HV', 'VH', 'VV'], ll=None, ur=None):
    path0 = os.path.dirname(fnann)
    annotation = UAVSARAnnotation(fnann)
    im = []
    for pol in polarizations:
        im.append(UAVSARReadRawImage(os.path.join(path0, annotation['slc' + pol].value),
                                     fnann, ll=ll, ur=ur))
    im = np.array(im)
    im = np.moveaxis(im, 0, -1)
    return im, annotation

def UAVSARAnnotation(fnann):
    annotationentry = namedtuple('annotationentry', ['unit', 'comment', 'value'])
    with open(fnann, 'r') as annFile:
        annotation = {}
        line0 = annFile.readline()
        annotation['id'] = line0.split(' ')[-1].strip()
        for line in annFile:
            try:
                assert line[0] != ';'
                linesplit = line.split('=')
                assert len(linesplit) >= 2
                if linesplit[0].count('(') >= 1 and linesplit[0].count(')') >= 1:
                    descrsplit = linesplit[0].rsplit('(', maxsplit=1)
                    descr = descrsplit[0].strip()
                    unit = descrsplit[1].split(')')[0]
                else:
                    descr = linesplit[0].strip()
                    unit = None
                linesplit2 = linesplit[1].split(';')
                if len(linesplit2) == 2:
                    comment = linesplit2[1].strip()
                else:
                    comment = None
                valuestr = linesplit2[0].strip()
                value = None
                usevaluestr = False
                if valuestr.count('.') == 1:
                    try:
                        value = float(valuestr)
                    except:
                        usevaluestr = True
                else:
                    try:
                        value = int(valuestr)
                    except:
                        usevaluestr = True
                value = valuestr if usevaluestr else value
                # value = valuestr
                annotation[descr] = annotationentry(value=value, unit=unit, comment=comment)
            except:
                pass
    return annotation

def UAVSARReadRawImage(fn, fnann, imagetype=None, ll=None, ur=None):
    annotation = UAVSARAnnotation(fnann)
    if os.path.splitext(fn)[1] in ['.slc']:
        if imagetype is None:
            rows = annotation['slc_mag.set_rows'].value
            cols = annotation['slc_mag.set_cols'].value
        else:
            rows = annotation[imagetype + ' Rows'].value
            cols = annotation[imagetype + ' Columns'].value
#         print(os.path.basename(fn[-9:-4]))
#         print(rows, cols)
        dtype = '<c8'
    elif os.path.splitext(fn)[1] in ['.mlc']:
        rows = annotation['mlc_mag.set_rows'].value
        cols = annotation['mlc_mag.set_cols'].value
        if 'HHHV' in fn or 'HHVV' in fn or 'HVVV' in fn:
            dtype = '<c8'
        else:
            dtype = '<f4'
    elif os.path.splitext(fn)[1] in ['.inc']:
        rows = annotation['grd_mag.set_rows'].value
        cols = annotation['grd_mag.set_cols'].value
        dtype = '<f4'
    im = np.fromfile(fn, dtype=dtype).reshape((rows, cols))
#     im = np.reshape(im, (rows, cols))
    if ll is not None and ur is not None:
        im = im[ll[0]:ur[0], ll[1]:ur[1]].copy()
    return im

def UAVSARReadQuadPolStack(
        imagenames, pathuav='~', ll=None, ur=None, polarizations=['HH', 'HV', 'VH', 'VV'], 
        segment=1, cmlversion='1x1'):
    stack = []
    for imagename in imagenames:
        for pol in polarizations:
            print(imagename, pol)
            listann = [fn for fn in os.listdir(pathuav) if (
                imagename in fn and fn[-4:] == '.ann' and fn.split('_')[-3][-2:] == pol)]
            assert len(listann) == 1
            fnann = os.path.join(pathuav, listann[0])
            annotation = UAVSARAnnotation(fnann)
            imagetype = f'slc_{segment}_{cmlversion}'
            fnim = os.path.join(pathuav, annotation[imagetype].value)
            im = UAVSARReadRawImage(fnim, fnann, imagetype, ll=ll, ur=ur)
            stack.append(im)
            print(im.shape)
    stack = np.array(stack)
    stack = np.moveaxis(stack, 0, -1)
    return stack

def coherence_2SLCs(cpx_im1,cpx_im2,cc_wind=(5,5)):
    '''
       coherence of two co-registered complex images
    '''
    coherence = np.zeros(cpx_im1.shape)
    intf_cpx = cpx_im1*np.conj(cpx_im2)
    im1_intensity = np.square(np.abs(cpx_im1))
    im2_intensity = np.square(np.abs(cpx_im2))
    intf_cpx_ave = ndimage.uniform_filter(intf_cpx.real, size=cc_wind) + 1j * ndimage.uniform_filter(intf_cpx.imag, size=cc_wind)
    im1_intensity_ave = ndimage.uniform_filter(im1_intensity,size=cc_wind)
    im2_intensity_ave = ndimage.uniform_filter(im2_intensity,size=cc_wind)
    coherence = np.abs(intf_cpx_ave*cc_wind[0]*cc_wind[1])/np.sqrt((im1_intensity_ave*cc_wind[0]*cc_wind[1])*(im2_intensity_ave*cc_wind[0]*cc_wind[1]))
    return coherence

def phase_2SLCs(cpx_im1,cpx_im2):
    '''
        Interferometric phase of coregistered complex images cpx_im1 and cpx_im2
    '''
    phase = np.angle( cpx_im1*np.conj(cpx_im2) )
    return phase

